In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
from sklearn import tree
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import scale
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
import functions as ff
from functions import *

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
X_train = df_train.drop(columns=['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
X_test = df_test.drop(columns=['id'])
Y_train = df_train[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

In [3]:
ff.transformacion(X_train)
ff.transformacion(X_test)

In [4]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_train, Y_train, test_size=0.2)

In [6]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [9]:
params={'ccp_alpha':[0.0,0.1,0.2],
        'max_depth':[4,6,8],
        'max_features': [0.3,0.5,0.7],
        'min_samples_split': [4,6,8],
        'min_samples_leaf': [5, 10, 15],
        'n_estimators': [20, 35, 50]
        }

scoring = ['accuracy', 'roc_auc']

n_cv=4 

grid_solver = GridSearchCV(estimator = clf,
                   param_grid = params, 
                   scoring = scoring,
                   cv = n_cv,
                   refit = 'roc_auc',
                   verbose = 1)

In [10]:
model = grid_solver.fit(X_train_t, y_train_t)

Fitting 4 folds for each of 729 candidates, totalling 2916 fits


In [11]:
model.best_params_

{'ccp_alpha': 0.0,
 'max_depth': 8,
 'max_features': 0.3,
 'min_samples_leaf': 10,
 'min_samples_split': 6,
 'n_estimators': 50}

In [16]:
best_model=model.best_estimator_
importances=pd.DataFrame([X_test.columns,best_model.feature_importances_], index=["feature","importance"]).T
print(importances.sort_values("importance", ascending = False).head(15))

                  feature importance
17        Outside_X_Index   0.176445
27               X_Region   0.131078
22            Log_X_Index   0.106059
21             LogOfAreas   0.085047
4            Pixels_Areas   0.059344
13  Steel_Plate_Thickness   0.052064
10     Length_of_Conveyer   0.045691
7       Sum_of_Luminosity   0.041255
35              Total_Log   0.035987
24      Orientation_Index   0.024393
11       TypeOfSteel_A300   0.021559
6             Y_Perimeter   0.018297
16           Square_Index   0.016874
28                 X_Mean    0.01593
19          Edges_Y_Index   0.014161


In [17]:
yhat_test_t=best_model.predict(X_test_t)
print('Roc Auc Tree:', roc_auc_score(y_true=y_test_t, y_score=yhat_test_t))
print('Accuracy:', accuracy_score(y_true=y_test_t, y_pred=yhat_test_t))

Roc Auc Tree: 0.6524284076948377
Accuracy: 0.31815816857440166


In [19]:
yhat_train_t=best_model.predict(X_train_t)
print('Roc Auc Tree:', roc_auc_score(y_true=y_train_t, y_score=yhat_train_t))
print('Accuracy:', accuracy_score(y_true=y_train_t, y_pred=yhat_train_t))

Roc Auc Tree: 0.6610289257880722
Accuracy: 0.3322926829268293


In [20]:
final_model = best_model.fit(X_train, Y_train)

In [21]:
y_pred_test=np.array(final_model.predict_proba(X_test))[:,:,1].T

In [22]:
submission=pd.read_csv("sample_submission.csv")
submission.iloc[:,1:8]=y_pred_test
submission.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.359667,0.018792,0.009506,0.000611,0.045343,0.163831,0.359485
1,19220,0.233941,0.037323,0.016870,0.003355,0.132703,0.166244,0.364935
2,19221,0.039414,0.036423,0.047251,0.021695,0.021288,0.312056,0.461868
3,19222,0.096070,0.041043,0.006809,0.003134,0.011378,0.411482,0.394146
4,19223,0.035047,0.029659,0.010201,0.009426,0.016762,0.432690,0.416747


In [23]:
submission.to_csv("9th_predict_submission.csv", index = False)